# Deploy a machine learning model on sagemaker

The objective of this workshop is to understand the steps needed to deploy a machine learning model on SageMaker.
To do that, it is necessary to understand how SageMaker works, and what it needs to work properly.

For that, we will understand how to preprocess the data, and how to properly parameter the SageMaker sdk functions.

Objectives :
- Understand the problem
- Understand how SageMaker works
- Preprocess the data in a manner that works with SageMaker
- Understand how to use hyperparameter optimisation with SageMaker to get the best model
- Understand the steps to deploy a model with SageMaker

In [32]:
import sagemaker

sess = sagemaker.Session()
#bucket = "s3://sagemaker-data-workshop"
bucket_name=boto3.client("s3").list_buckets()["Buckets"][0]["Name"]
bucket="s3://"+bucket_name

# Define IAM role
import boto3
from sagemaker import get_execution_role

role = get_execution_role()

In [21]:
import pandas as pd
import numpy as np
import io
import os
import sys
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

from sagemaker.predictor import Predictor


# <center> Plan of actions
![Plan of actions](Workflow-diagram.jpg)


## Preprocessing the data

Important notions to keep in mind :
- The model will only accept data without headers, so we use numpy instead of pandas
- The target of the model needs to be the first column (sagemaker default)

The dataset is available from UCI Machine Learning. The aim for this task is to determine age of an Abalone (a kind of shellfish) from its physical measurements. At the core, it's a regression problem. The dataset contains several features - sex (categorical), length (continuous), diameter (continuous), height (continuous), whole_weight (continuous), shucked_weight (continuous), viscera_weight (continuous), shell_weight (continuous) and rings (integer).Our goal is to predict the variable rings which is a good approximation for age (age is rings + 1.5).



In [3]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Since we get a headerless CSV file we specify the column names here.
feature_columns_names = [
    'sex', # M, F, and I (infant)
    'length', # Longest shell measurement
    'diameter', # perpendicular to length
    'height', # with meat in shell
    'whole_weight', # whole abalone
    'shucked_weight', # weight of meat
    'viscera_weight', # gut weight (after bleeding)
    'shell_weight'] # after being dried

label_column = "rings"

feature_columns_dtype = {
    "sex": str,
    "length": np.float64,
    "diameter": np.float64,
    "height": np.float64,
    "whole_weight": np.float64,
    "shucked_weight": np.float64,
    "viscera_weight": np.float64,
    "shell_weight": np.float64,
}

label_column_dtype = {'rings': "float64"} # +1.5 gives the age in years

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z


if __name__ == "__main__":


    df = pd.read_csv(
        bucket+"/abalone.csv",
        header=None,
        names=feature_columns_names + [label_column],
        dtype=merge_two_dicts(feature_columns_dtype, label_column_dtype),
        )
    
    numeric_features = list(feature_columns_names)
    numeric_features.remove("sex")
    numeric_transformer = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
                                  )

    categorical_features = ["sex"]
    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
                                      )

    preprocess = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ]
                                  )
    
    train, validation, test = np.split(df, [int(0.7 * len(df)), int(0.85 * len(df))])
    
    y = train.pop("rings")
    X_pre = preprocess.fit_transform(train)
    y_pre = y.to_numpy().reshape(len(y), 1)

    train = np.concatenate((y_pre, X_pre), axis=1)
    
    y = validation.pop("rings")
    X_pre = preprocess.transform(validation)
    y_pre = y.to_numpy().reshape(len(y), 1)

    validation = np.concatenate((y_pre, X_pre), axis=1)
    

    y = test.pop("rings")
    X_pre = preprocess.transform(test)
    y_pre = y.to_numpy().reshape(len(y), 1)

    test = np.concatenate((y_pre, X_pre), axis=1)
    

    
    pd.DataFrame(train).to_csv("train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(
        "validation.csv", header=False, index=False
                                   )
    pd.DataFrame(test).to_csv("test.csv", header=False, index=False)

In [4]:
boto3.Session().resource("s3").Bucket(bucket_name).Object( #Upload train data to s3
    ("train/train.csv")
).upload_file("train.csv")

boto3.Session().resource("s3").Bucket(bucket_name).Object( #Upload validation data to s3
    ("validation/validation.csv")
).upload_file("validation.csv")

boto3.Session().resource("s3").Bucket(bucket_name).Object( #Upload test data to s3
    ("test/test.csv")
).upload_file("test.csv")

## Hyperparameter tuning with xgboost

In [36]:
# Each step in SageMaker has its own input : Traininginput for training, transforminput for batch_transform,
# ProcessingInput for processing, etc...

s3_input_train = TrainingInput( 
    s3_data="{}/train/train.csv".format(bucket), content_type="csv"
)
s3_input_validation = TrainingInput(
    s3_data="{}/validation/validation.csv".format(bucket), content_type="csv"
)


- Load the image that runs inside the EC2 machine that trains the model
- Specify the parameters of the models, and the parameters that we are going to tune

In [37]:
container = sagemaker.image_uris.retrieve("xgboost",
                                          boto3.Session().region_name,
                                          "latest")

In [38]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path="{}/output".format(bucket),
    sagemaker_session=sess,
)

xgb.set_hyperparameters(
    eval_metric="rmse",
    objective="reg:linear",
    num_round=50, #Number of trees (num_estimator)
)

hyperparameter_ranges = {
    "alpha": ContinuousParameter(0.01, 10, scaling_type="Logarithmic"), #Parameter for L1 regularization
    "lambda": ContinuousParameter(0.01, 10, scaling_type="Logarithmic"), #Parameter for L2 regularization
    "max_depth": IntegerParameter(1, 10),
}

Types of [EC2 Machines](https://aws.amazon.com/ec2/instance-types/?trkCampaign=acq_paid_search_brand&sc_channel=PS&sc_campaign=acquisition_IBERIA&sc_publisher=Google&sc_category=Cloud%20Computing&sc_country=IBERIA&sc_geo=EMEA&sc_outcome=acq&sc_detail=amazon%20ec2%20instance&sc_content=%7Bad%20group%7D&sc_matchtype=e&sc_segment=536392708261&sc_medium=ACQ-P%7CPS-GO%7CBrand%7CDesktop%7CSU%7CCloud%20Computing%7CEC2%7CIBERIA%7CEN%7CSitelink&s_kwcid=AL!4422!3!536392708261!e!!g!!amazon%20ec2%20instance&ef_id=EAIaIQobChMI38OXroWC9gIVSgKLCh3pDwytEAAYASABEgJuvfD_BwE:G:s&s_kwcid=AL!4422!3!536392708261!e!!g!!amazon%20ec2%20instance) , their specs and their characteristics

In [39]:
objective_metric_name = "validation:rmse"
tuner = HyperparameterTuner(
    xgb,                     
    objective_metric_name,   
    hyperparameter_ranges,
    max_jobs=3,
    max_parallel_jobs=3,
    strategy="Random",         #Random hyperparameter search. Other alternative is Bayesian
                               #SageMaker doesn't support gridsearch
    objective_type="Minimize", #We want to minimize RMSE. If it was precision, we would have "Maximize"
    base_tuning_job_name="T1"  #Name for the tuning job
)

tuner.fit({"train": s3_input_train, "validation": s3_input_validation})

........................................!


## Tuning analytics

In [41]:
sage_client = boto3.client('sagemaker')

In [42]:
tuning_job_name=tuner.latest_tuning_job.job_name

In [43]:
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)


job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

is_minimize = objective["Type"] != "Maximize"

3 training jobs have completed


In [44]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", None)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 3
{'lowest': 2.19281005859375, 'highest': 2.343100070953369}


,alpha,lambda,max_depth,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
1,3.524426,0.057152,4.0,T1-220216-1724-002-130f1969,Completed,2.19281,2022-02-16 17:26:33+00:00,2022-02-16 17:27:21+00:00,48.0
0,1.027243,4.621644,9.0,T1-220216-1724-003-55e12879,Completed,2.32833,2022-02-16 17:26:28+00:00,2022-02-16 17:27:11+00:00,43.0
2,0.126621,0.171751,9.0,T1-220216-1724-001-b6aff576,Completed,2.34310,2022-02-16 17:26:22+00:00,2022-02-16 17:27:03+00:00,41.0


In [45]:
if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    print(tuning_job_result["BestTrainingJob"])

Best model found so far:
{'TrainingJobName': 'T1-220216-1724-002-130f1969', 'TrainingJobArn': 'arn:aws:sagemaker:eu-west-1:775923162736:training-job/t1-220216-1724-002-130f1969', 'CreationTime': datetime.datetime(2022, 2, 16, 17, 24, 20, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2022, 2, 16, 17, 26, 33, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2022, 2, 16, 17, 27, 21, tzinfo=tzlocal()), 'TrainingJobStatus': 'Completed', 'TunedHyperParameters': {'alpha': '3.524426056570139', 'lambda': '0.05715202020575164', 'max_depth': '4'}, 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:rmse', 'Value': 2.19281005859375}, 'ObjectiveStatus': 'Succeeded'}


In [46]:
tuning_job_result["BestTrainingJob"]["TunedHyperParameters"]

{'alpha': '3.524426056570139',
 'lambda': '0.05715202020575164',
 'max_depth': '4'}

In [47]:
best_params=tuning_job_result["BestTrainingJob"]["TunedHyperParameters"]

### Building an xgboost model with best parameters and deploying it

In [48]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path="{}/output".format(bucket),
    sagemaker_session=sess,
)

xgb.set_hyperparameters(
    eval_metric="rmse",
    objective="reg:linear",
    num_round=50,
    
    #Getting the best params
    alpha=best_params["alpha"],
    reg_lambda=best_params["lambda"],
    max_depth=best_params["max_depth"]   
)


In [49]:
xgb.fit({"train": s3_input_train}) # Put the whole data instead of just train

2022-02-16 17:29:19 Starting - Starting the training job...
2022-02-16 17:29:21 Starting - Launching requested ML instancesProfilerReport-1645032559: InProgress
......
2022-02-16 17:30:48 Starting - Preparing the instances for training......
2022-02-16 17:31:44 Downloading - Downloading input data...
2022-02-16 17:32:17 Training - Training image download completed. Training in progress.
2022-02-16 17:32:17 Uploading - Uploading generated training modelArguments: train
[2022-02-16:17:32:12:INFO] Running standalone xgboost training.
[2022-02-16:17:32:12:INFO] Path /opt/ml/input/data/validation does not exist!
[2022-02-16:17:32:12:INFO] File size need to be processed in the node: 0.43mb. Available memory size in the node: 8009.94mb
[2022-02-16:17:32:12:INFO] Determined delimiter of CSV input is ','
[17:32:12] S3DistributionType set as FullyReplicated
[17:32:12] 2923x10 matrix with 29230 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[17:32:12] src/tree/


Now we are going to host an endpoint, we can make real-time predictions from our model very easily, simply by making a http POST request. But first, we'll need to set up serializers and deserializers for passing our test_data NumPy arrays to the model behind the endpoint.

Now, we'll use a simple function to:

- Loop over our test dataset <br>
- Split it into mini-batches of rows <br> 
- Convert those mini-batchs to CSV string payloads <br>
- Retrieve mini-batch predictions by invoking the XGBoost endpoint <br>
- Collect predictions and convert from the CSV output our model provides into a NumPy array <br>


In [63]:
model_description=xgb.jobs[0].describe()
model_path=os.path.join(model_description["OutputDataConfig"]["S3OutputPath"],
                         model_description["TrainingJobName"],
                         "output",
                         "model.tar.gz"
                       )
model_path

's3://sagemaker-data-workshop/output/xgboost-2022-02-16-17-29-19-095/output/model.tar.gz'

In [64]:
xgb_model=sagemaker.Model(image_uri=container,model_data=model_path,role=role)

In [65]:
xgb_model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge', endpoint_name="xgb-endpoint")

----!

Serialization is the process of converting a data object—a combination of code and data represented within a region of data storage—into a series of bytes that saves the state of the object in an easily transmittable form

In [66]:
test=pd.read_csv(f"s3://{bucket_name}/test/test.csv")
predictor=Predictor(endpoint_name="xgb-endpoint",
                    sagemaker_session=sess,
                    serializer=CSVSerializer()  
)


In [67]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1)) #Mini batch
    predictions = ""
    for array in split_array:
        predictions = ",".join([predictions, predictor.predict(array).decode("utf-8")]) #Make predictions and 
                                                                                        #Desereliaze data

    return np.fromstring(predictions[1:], sep=",")


predictions = predict(test.to_numpy()[:, 1:]) #remove the target


In [68]:
len(predictions)

626

In [69]:
test.shape

(626, 11)

In [70]:
predictor.delete_endpoint()

### Batch transform

Use batch transform when you need to do the following:

- Preprocess datasets to remove noise or bias that interferes with training or inference from your dataset.
- Get inferences from large datasets.
- Run inference when you don't need a persistent endpoint.
- Associate input records with inferences to assist the interpretation of results.


In [71]:
transformer=xgb_model.transformer(instance_count=1,
    instance_type="ml.m5.xlarge",
    strategy="MultiRecord",
    assemble_with="Line",
    accept="text/csv",
    output_path=f"s3://{bucket_name}/predictions",
                                )

# Strategy
Specifies the number of records to include in a mini-batch for an HTTP inference request. A record is a single unit of input data that inference can be made on. For example, a single line in a CSV file is a record.

To enable the batch strategy, you must set the SplitType property to Line, RecordIO, or TFRecord.

To use only one record when making an HTTP invocation request to a container, set BatchStrategy to SingleRecord and SplitType to Line.

To fit as many records in a mini-batch as can fit within the MaxPayloadInMB limit, set BatchStrategy to MultiRecord and SplitType to Line.



In [74]:
transformer.transform(f"s3://{bucket_name}/test/test.csv",
                      content_type="text/csv",
                      join_source="Input",
                      split_type="Line",
                      input_filter="$[1:]", #remove first column since the model isn't, and shouldn't be fitted
                                            #with the targets 
                      output_filter="$[0,-1]") #returns first and last column, which are targets and predictions


.........................Arguments: serve
Arguments: serve
[2022-02-16 18:15:01 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2022-02-16 18:15:01 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2022-02-16 18:15:01 +0000] [1] [INFO] Using worker: gevent
[2022-02-16 18:15:01 +0000] [21] [INFO] Booting worker with pid: 21
[2022-02-16 18:15:01 +0000] [22] [INFO] Booting worker with pid: 22
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (/opt/amazon/lib/python3.7/site-packages/urllib3/util/__init__.py)', 'urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)']. 
  monkey.patch_all(subprocess=True)


In [75]:
pd.read_csv(f"s3://{bucket_name}/predictions/test.csv.out",names=["Target","Prediction"])

,Target,Prediction
0,8.0,8.439944
1,9.0,8.267351
2,8.0,8.052462
3,9.0,8.496286
4,8.0,9.531392
...,...,...
622,11.0,9.841301
623,10.0,10.345934
624,9.0,11.895207
625,10.0,9.898403
